In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf

from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from keras.models import load_model
best_model = load_model('bestModel.h5')

#import tensorflow as tf

#converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
#tflite_model = converter.convert()
# Save the converted model to a file
#with open('model.tflite', 'wb') as f:
#    f.write(tflite_model)

# Load the TFLite model
#interpreter = tf.lite.Interpreter(model_path='bestModel.tflite')
#interpreter.allocate_tensors()

# Print model details
#print("TFLite model loaded and allocated tensors.")

INFO:tensorflow:Assets written to: C:\Users\mosta\AppData\Local\Temp\tmp8vftxhio\assets


INFO:tensorflow:Assets written to: C:\Users\mosta\AppData\Local\Temp\tmp8vftxhio\assets


TFLite model loaded and allocated tensors.


In [3]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')

lbl=['Close','Open']

model = load_model('bestModel.h5')
path = os.getcwd()
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('Speed.mp4')

# Set resolution and frame rate
cap.set(cv2.CAP_PROP_FPS, 60)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 150)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 150)

font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(150,150))
    left_eye = leye.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(50,50))
    right_eye =  reye.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(50,50))


    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(64,64))
        r_eye=  r_eye.reshape((-1, 64, 64, 1))
        r_eye = r_eye/255.0

        rpred = model.predict(r_eye)
        print(rpred)
        if(rpred[0]>0.5):
            lbl='Open' 
        if(rpred[0]<=0.5):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(64,64))
        l_eye=l_eye.reshape((-1, 64, 64, 1))
        l_eye = l_eye/255.0
        
        lpred = model.predict(l_eye)
        print(lpred)
        if(lpred[0]>0.5):
            lbl='Open'   
        if(lpred[0]<=0.5):
            lbl='Closed'
        break
    

    if(rpred[0]<=0.5 and lpred[0]<=0.5):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>12):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        #try:
        #    sound.play()
            
        #except:  # isplaying = False
        #    pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

pygame 2.6.0 (SDL 2.28.4, Python 3.8.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'cap' is not defined